In [5]:
from transformers import AutoTokenizer, AutoModel

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Specify the directory path where you want to save the model and tokenizer
save_directory = "/Users/david/Desktop/QA Media/Models"

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


('/Users/david/Desktop/QA Media/Models/tokenizer_config.json',
 '/Users/david/Desktop/QA Media/Models/special_tokens_map.json',
 '/Users/david/Desktop/QA Media/Models/vocab.txt',
 '/Users/david/Desktop/QA Media/Models/added_tokens.json',
 '/Users/david/Desktop/QA Media/Models/tokenizer.json')

In [6]:
#Import Modules
import requests
from bs4 import BeautifulSoup
import pandas as pd

articles = {}

# Load the Excel file
df = pd.read_excel('article urls.xlsx')

main_url = "https://www.qa-financial.com"

# Convert the DataFrame to a list
urls = df['URL'].tolist()  # Replace 'urls' with the actual column name in the Excel file

# For each URL, get the text content
for url in urls:
    try:
        response = requests.get(main_url + url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get the title and text based on your description
        #title = soup.find('title').get_text()
        text = soup.find('meta', attrs={'name': 'description'}).get('content')

        articles[main_url + url] = text
    except:
        continue
        
import json

# Save the dictionary to a file
with open('dict_file.json', 'w') as file:
    json.dump(articles, file)

In [7]:
article_content = [x for x in articles.values()]

# Function to get embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

# Assume `documents` is a list of your news articles
embeddings = [get_embeddings(doc) for doc in article_content]

In [8]:
import pickle

# Save the list of arrays.
with open('arrays.pkl', 'wb') as f:
    pickle.dump(embeddings, f)
